In [1]:
from darts import TimeSeries
from darts.models import RNNModel
from darts.utils.data import TimeseriesDataset
from darts.metrics import mape
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from sklearn.preprocessing import MinMaxScaler

# Load data
df = pd.read_csv('nifty_dataPEE.csv')
series = TimeSeries.from_dataframe(df, 'lasttradetime', ['close'])

# Split data
train, val = series.split_before(pd.Timestamp('2024-01-01'))

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)

# Define model parameters
lookback_window = 5
batch_size = 30
units = 100
epochs = 20

# Create dataset
train_dataset = TimeseriesDataset.from_series(train_scaled, lookback_window + 1)
val_dataset = TimeseriesDataset.from_series(val_scaled, lookback_window + 1)

# Create and train the model
model = RNNModel(
    model='LSTM',
    input_chunk_length=lookback_window,
    output_chunk_length=1,
    hidden_size=units,
    n_rnn_layers=1,
    batch_size=batch_size,
    n_epochs=epochs,
    optimizer_kwargs={'lr': 1e-3},
    model_name='LSTM_nifty',
    log_tensorboard=True,
    random_state=0
)

model.fit(train_dataset, val_series=val_scaled, verbose=True)

# Make forecasts
forecast = model.predict(n=len(val))

# Inverse scaling
forecast = scaler.inverse_transform(forecast)

# Print max and min forecasted close prices
print(forecast['close'].max(), forecast['close'].min())


ImportError: cannot import name 'TimeseriesDataset' from 'darts.utils.data' (c:\Users\santh\anaconda3\Lib\site-packages\darts\utils\data\__init__.py)

In [3]:
import pandas as pd
import numpy as np

In [4]:
from darts import TimeSeries
from darts.models import RNNModel
from darts.metrics import mape
from sklearn.preprocessing import MinMaxScaler

# Load data
df = pd.read_csv('nifty_dataPEE.csv')
series = TimeSeries.from_dataframe(df, 'lasttradetime', ['close'])

# Split data
train, val = series.split_before(pd.Timestamp('2024-01-01'))

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)

# Define model parameters
lookback_window = 5
batch_size = 30
units = 100
epochs = 20

# Convert time series data into sequences
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1):
        X.append(data[i:(i + lookback_window)])
        y.append(data[i + lookback_window])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled, lookback_window)
X_val, y_val = create_sequences(val_scaled, lookback_window)

# Create and train the model
model = RNNModel(
    model='LSTM',
    input_chunk_length=lookback_window,
    output_chunk_length=1,
    hidden_size=units,
    n_rnn_layers=1,
    batch_size=batch_size,
    n_epochs=epochs,
    optimizer_kwargs={'lr': 1e-3},
    model_name='LSTM_nifty',
    log_tensorboard=True,
    random_state=0
)

model.fit(X_train, y_train, verbose=True, val_series=X_val, val_targets=y_val)

# Make forecasts
forecast = model.predict(n=len(val_scaled))

# Inverse scaling
forecast = scaler.inverse_transform(forecast)

# Print max and min forecasted close prices
print(forecast['close'].max(), forecast['close'].min())


ValueError: The time index of the provided DataArray is missing the freq attribute, and the frequency could not be directly inferred. This probably comes from inconsistent date frequencies with missing dates. If you know the actual frequency, try setting `fill_missing_dates=True, freq=actual_frequency`. If not, try setting `fill_missing_dates=True, freq=None` to see if a frequency can be inferred.


ValueError: The time index of the provided DataArray is missing the freq attribute, and the frequency could not be directly inferred. This probably comes from inconsistent date frequencies with missing dates. If you know the actual frequency, try setting `fill_missing_dates=True, freq=actual_frequency`. If not, try setting `fill_missing_dates=True, freq=None` to see if a frequency can be inferred.